**Workflow**:

---

1. Import Packages
2. STL Instantiation
3. STL Actualized Seasonalities
4. FB Instantiation
5. FB Actualized Seasonalities (For comparison)

In [3]:
#Import Packages
import pandas as pd
from prophet import Prophet
from google.colab import files
import xlrd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.seasonal import MSTL
!pip install TBATS

from tbats import TBATS



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 23.4 MB/s eta 0:00:00


In [ ]:
#STL Instantiation

#1. Import
df = pd.read_excel('/content/CA Recharge.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

#2. Data Cleaning

df = df.set_index(['ds'])
df = df.asfreq('D')
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] =df['y'].interpolate()
df['y'] = df['y'].ffill().bfill()
df= df.sort_index()
print(df['y'].isna().sum())

#3. Weekly Seasonality fitting

act_weekly_seasonality = STL(df['y'], period=7)
act_weekly_model = act_weekly_seasonality.fit()
act_weekly_excel = pd.DataFrame({'date': df.index, 'y':act_weekly_model.observed, 'seasonal':act_weekly_model.seasonal})
act_weekly_excel.to_excel("act_weekly.xlsx")


#4. Monthly Seasonality fitting
act_monthly_seasonality = STL(df['y'], period=30)
act_monthly_model = act_monthly_seasonality.fit()
act_monthly_excel = pd.DataFrame({'date': df.index, 'y':act_monthly_model.observed, 'seasonal':act_monthly_model.seasonal})
act_monthly_excel = act_monthly_excel[(act_monthly_excel['date'].dt.month == 12) & (act_monthly_excel['date'].dt.year == 2024)]
act_monthly_excel.to_excel("act_monthly.xlsx")




0


In [ ]:
#STL v Prophet Sanity Check

#1. STL Weekly Actualized Seasonality
weekly_decompose_df = act_weekly_model.seasonal.to_frame('seasonal')
weekly_decompose_df['weekday'] = weekly_decompose_df.index.day_name()
weekly_summary = weekly_decompose_df.groupby('weekday')['seasonal'].mean()
weekly_summary.to_excel('STL_weekly_summary.xlsx')

#2. STL Monthly Actualized Seasonality
monthly_decompose_df = act_monthly_model.seasonal.to_frame('seasonal')
monthly_decompose_df['day_of_month'] = monthly_decompose_df.index.day
monthly_summary = monthly_decompose_df.groupby('day_of_month')['seasonal'].mean()
monthly_summary.to_excel('STL_monthly_summary.xlsx')

In [ ]:
# TBATS

#1. Import

df = pd.read_excel('/content/CA Recharge.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

y = df['y'].values

model = TBATS(use_box_cox = True, use_trend=True,use_damped_trend=True,use_arma_errors=True, seasonal_periods=[7,30,365])

model_fit= model.fit(y)
print(model_fit.summary())

In [8]:
#Output file

import numpy as np

raw_file = model_fit.seasonal_components_
seasonal_periods = model_fit.seasonal_periods
components = model_fit.components_per_period

seasonal_effects ={}

idx = np.cumsum([0]+list(components))

for i, p in enumerate(seasonal_periods):
    start, end = idx[i],idx[i+1]
    seasonal_effects[p] = raw_file[:,start:end].sum(axis=1)

name_map = { 7: 'weekly_effect', 30 : 'monthly_effect', 365: 'annual_effect'}

for p, values in seasonal_effects.items():
  df_new[name_map[p]] = values

df_new['tbats_trend']  = tbats_model.level
df_new['tbats_fitted'] = tbats_model.y_hat
df_new['tbats_resid']  = df-new['y'] - df_new['tbats_fitted']

# 5. OPTIONAL: Export
df_new.to_excel('TBATS_decomposition.xlsx', index=False)





AttributeError: 'Model' object has no attribute 'trend'

In [ ]:
#Fourier Instantiation

#1. Import
df_ca = pd.read_excel('/content/CA Recharge.xlsx')
df_ca['ds'] = pd.to_datetime(df_ca['ds'])

#2. Model fitting
m1 = Prophet(daily_seasonality=False, yearly_seasonality=False, seasonality_mode='additive')
m1.add_seasonality(name='weekly', period=7, fourier_order=3)
m1.add_seasonality(name='monthly', period=30, fourier_order=5)
m1.add_seasonality(name='annual', period=365, fourier_order=10)
m1.add_country_holidays(country_name='PAK')
m1.fit(df_ca)

#2. Forecast
df_ca_future = m1.make_future_dataframe(periods=7)
forecast_ca = m1.predict(df_ca_future)
forecast_ca.to_excel('forecast_ca.xlsx')







INFO:prophet:Found custom seasonality named 'weekly', disabling built-in 'weekly' seasonality.


In [ ]:
#Fourier Actualized Seasonalities

#1. Weekly actualized Seasonality
weekly_decompose_df2 = forecast_ca[['ds','yhat','weekly']].copy()
weekly_decompose_df2['weekday'] = weekly_decompose_df2['ds'].dt.day_name()
weekly_summary = weekly_decompose_df2.groupby('weekday')['weekly'].mean()
weekly_summary.to_excel('FB_weekly_summary.xlsx')

#2. Monthly Actualized Seasonality
monthly_decompose_df2 = forecast_ca[['ds','yhat','monthly']].copy()
monthly_decompose_df2['day_of_month'] = monthly_decompose_df2['ds'].dt.day
#monthly_summary = monthly_decompose_df2.groupby('day_of_month')['monthly'].mean()
monthly_summary = monthly_decompose_df2[(monthly_decompose_df2['ds'].dt.month == 12) & (monthly_decompose_df2['ds'].dt.year == 2024)]
monthly_summary = monthly_summary.groupby('day_of_month')['monthly'].mean()
monthly_summary.to_excel('FB_monthly_summary.xlsx')

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

df_cv = cross_validation(m, initial = '540 days', period = '30 days', horizon = '30 days')

df_p = performance_metrics(df_cv)
df_p.to_excel('df_p.xlsx')
df_cv.to_excel('df_cv.xlsx')

In [ ]:
from google.colab import drive

forecast.to_excel('/content/forecast.xlsx', index=False)

In [ ]:
!git clone https://github.com/Sannan27/glowing-adventure.git

%cd glowing-adventure

!git config user.name "abc"
!git config user.email "abc"

!git add .
!git commit -m"Recharge Forecast v 1.0"
!git push

In [ ]:
%cd /content
!rm -rf glowing-adventure

/content


In [30]:
#MSTL
import numpy as np

df = pd.read_excel('/content/CA Recharge.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

#Canvas

df = df.set_index(['ds'])
df = df.asfreq('D')
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] =df['y'].interpolate()
df['y'] = df['y'].ffill().bfill()
df= df.sort_index()


#Model instantiation

mstl = MSTL(df['y'], periods=[7,30,365])
mstl_re = mstl.fit()
#mstl_df = pd.DataFrame(mstl_re.seasonal, index = df.index, columns=['weekly'])



seasonal = mstl_re.seasonal  # shape (T, 3)
trend    = mstl_re.trend     # shape (T,)
resid    = mstl_re.resid     # shape (T,)

# Assign by POSITION, not by index
seasonal = np.asarray(mstl_re.seasonal)
trend    = np.asarray(mstl_re.trend)
resid    = np.asarray(mstl_re.resid)

print("seasonal shape:", seasonal.shape)
print("len(df):", len(df))

df['weekly_mstl']  = seasonal[:, 0]
df['monthly_mstl'] = seasonal[:, 1]
df['annual_mstl']  = seasonal[:, 2]

df['trend_mstl'] = trend
df['resid_mstl'] = resid

print(df[['weekly_mstl', 'monthly_mstl', 'annual_mstl']].head())
print(df[['weekly_mstl', 'monthly_mstl', 'annual_mstl']].describe())

df.to_excel('df_mstl.xlsx')

#weekly_seasonality = mstl_df['weekly'].to_frame('seasonality_index')
#weekly_seasonality['weekday'] = weekly_seasonality.index.day_name()
#weekly_seasonality.to_excel('test.xlsx')
#weekly_seasonality = weekly_seasonality.dropna()

#weekly_summary = weekly_seasonality.groupby('weekday')['seasonality_index'].mean()
#print(weekly_seasonality.isna().sum())







seasonal shape: (1070, 3)
len(df): 1070
             weekly_mstl  monthly_mstl   annual_mstl
ds                                                  
2023-01-01 -2.136045e+06  1.683038e+06  3.398992e+05
2023-01-02  1.851284e+06  1.830575e+06 -1.415877e+06
2023-01-03  7.504490e+05  1.732260e+06 -9.492377e+05
2023-01-04  2.728950e+05  1.569363e+06 -4.188399e+05
2023-01-05  2.038931e+05  7.386066e+05 -2.267436e+05
        weekly_mstl  monthly_mstl   annual_mstl
count  1.070000e+03  1.070000e+03  1.070000e+03
mean  -6.815759e+02  7.808366e+03 -1.308234e+05
std    1.411467e+06  1.330815e+06  1.390524e+06
min   -4.173553e+06 -3.200414e+06 -9.267704e+06
25%   -8.494085e+05 -9.683514e+05 -8.464913e+05
50%    2.307982e+05 -3.761065e+05 -6.824627e+04
75%    8.938677e+05  1.019790e+06  6.558046e+05
max    3.361300e+06  3.571710e+06  5.707154e+06


In [26]:
seasonal = mstl_re.seasonal
print("seasonal shape:", seasonal.shape)
print("any NaNs in seasonal:", np.isnan(seasonal).sum())

print(df[['weekly_mstl', 'monthly_mstl', 'annual_mstl']].head())
print(df[['weekly_mstl', 'monthly_mstl', 'annual_mstl']].describe())

seasonal shape: (1070, 3)
any NaNs in seasonal: seasonal_7      0
seasonal_30     0
seasonal_365    0
dtype: int64
            weekly_mstl  monthly_mstl  annual_mstl
ds                                                
2023-01-01          NaN           NaN          NaN
2023-01-02          NaN           NaN          NaN
2023-01-03          NaN           NaN          NaN
2023-01-04          NaN           NaN          NaN
2023-01-05          NaN           NaN          NaN
       weekly_mstl  monthly_mstl  annual_mstl
count          0.0           0.0          0.0
mean           NaN           NaN          NaN
std            NaN           NaN          NaN
min            NaN           NaN          NaN
25%            NaN           NaN          NaN
50%            NaN           NaN          NaN
75%            NaN           NaN          NaN
max            NaN           NaN          NaN
